### Importar librerías

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Importar dataset

In [2]:
df = pd.read_csv('data.csv')
print(df.columns)
df

Index(['Country', 'Age', 'Salary', 'Purchased'], dtype='object')


,Country,Age,Salary,Purchased
0,France,44.0,72000.0,No
1,Spain,27.0,48000.0,Yes
2,Germany,30.0,54000.0,No
3,Spain,38.0,61000.0,No
4,Germany,40.0,NaN,Yes
5,France,35.0,58000.0,Yes
6,Spain,NaN,52000.0,No
7,France,48.0,79000.0,Yes
8,Germany,50.0,83000.0,No
9,France,37.0,67000.0,Yes


### Definir las variables independientes y dependientes

In [3]:
# Use iloc to select the rows and columns by index and get the values as a numpy array
X = df.iloc[:, :-1].values # all rows (:), all columns except the last one (:-1)
y = df.iloc[:, -1].values # all rows (:), last column (-1) or (3) because there are 4 columns in total
print(X.shape, y.shape)

(10, 3) (10,)


### Tratar valores nulos

Se pueden tomar diferentes estrategias para manejar esta situación:
1. Eliminar las filas en las que se encuentran datos vacios. Una opción muy drástica y poco usada puesto que, al menos de que la fila venga vacía o casi, aún así aporta algo de información util para lo que estamos estudiando.
2. Sustituir los valores vacíos por medidas de localización de los datos, como la media, moda o mediana de la fila. Es la más usada y la que aprenderemos a hacer.

In [4]:
from sklearn.impute import SimpleImputer

# Verify if the data is clean and does not contain any null values
if df.isnull().values.any():
    nan_columns = df.isnull().any()
    nan_columns = {int(i) : col for i,col in zip(np.where(nan_columns)[0], df.columns[nan_columns].tolist())}
    print(f'Missing values in {nan_columns}')

imputer = SimpleImputer(missing_values=np.nan, strategy='mean') # Replace missing values with the mean of the column
imputer.fit(X[:, 1:3]) # Fit the imputer on the columns with missing values (1:3 means columns 1 and 2)
X[:, 1:3] = imputer.transform(X[:, 1:3]) # Transform the data and replace the missing values with the mean

pd.DataFrame(X)


Missing values in {1: 'Age', 2: 'Salary'}


,0,1,2
0,France,44.0,72000.0
1,Spain,27.0,48000.0
2,Germany,30.0,54000.0
3,Spain,38.0,61000.0
4,Germany,40.0,63777.777778
5,France,35.0,58000.0
6,Spain,38.777778,52000.0
7,France,48.0,79000.0
8,Germany,50.0,83000.0
9,France,37.0,67000.0


### Tratar con datos categoricos

Se hace cuando una determinada columna no contiene números, sino que tiene etiquetas o categorias de la fila en cuestion. Estas columnas se tratan de una manera en la que se pueda operar con ellas, es decir: codificandolas.

Para la primera columna (Country), en la que tenemos más de 2 categorias, usaremos la codificación **OneHot**, que lo que hace es separar dicha columna en n columnas (donde n es el numero de categorias en la columna) binarias en las que se pone un 1 si la columna correspondiente es la categoría de esa fila, un 0 en caso contrario. A esto también se le conoce como **variable dummy.**

In [40]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers= [('encoder', OneHotEncoder(), [0])], remainder='passthrough') # Apply one hot encoding to the first column (index 0) and keep the rest of the columns as they are
X = np.array(ct.fit_transform(X)) # Transform the data and convert it to a numpy array

pd.DataFrame(X)

,0,1,2,3,4,5
0,0.0,1.0,0.0,0.0,44.0,72000.0
1,1.0,0.0,0.0,1.0,27.0,48000.0
2,1.0,0.0,1.0,0.0,30.0,54000.0
3,1.0,0.0,0.0,1.0,38.0,61000.0
4,1.0,0.0,1.0,0.0,40.0,63777.777778
5,0.0,1.0,0.0,0.0,35.0,58000.0
6,1.0,0.0,0.0,1.0,38.777778,52000.0
7,0.0,1.0,0.0,0.0,48.0,79000.0
8,1.0,0.0,1.0,0.0,50.0,83000.0
9,0.0,1.0,0.0,0.0,37.0,67000.0


Para la variable dependiente $y$ (Purchased) usaremos la codificación **LabelEncoder**, que lo que hace es asignar un número a cada categoría de la columna. En este caso, como solo tenemos 2 categorías, no hay problema en usar esta codificación.

In [36]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder() # Create a label encoder object
y = le.fit_transform(y) # Fit the label encoder on the target variable and transform it
y

array([0, 1, 0, 0, 1, 1, 0, 1, 0, 1])

### Dividr el dataframe en conjuntos de entrenamiento y testeo

Se hace una división para estos dos conjuntos: entrenamiento y testeo, para evitar overfiting y poner a prueba el modelo con datos que no ha visto antes.

In [37]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state= 24) # Split the data into training and testing sets (80% train, 20% test) with a random state for reproducibility

X_train, X_test, y_train, y_test

(array([[0.0, 1.0, 0.0, 50.0, 83000.0],
        [1.0, 0.0, 0.0, 48.0, 79000.0],
        [1.0, 0.0, 0.0, 35.0, 58000.0],
        [0.0, 0.0, 1.0, 38.77777777777778, 52000.0],
        [0.0, 0.0, 1.0, 27.0, 48000.0],
        [1.0, 0.0, 0.0, 44.0, 72000.0],
        [0.0, 0.0, 1.0, 38.0, 61000.0],
        [0.0, 1.0, 0.0, 30.0, 54000.0]], dtype=object),
 array([[1.0, 0.0, 0.0, 37.0, 67000.0],
        [0.0, 1.0, 0.0, 40.0, 63777.77777777778]], dtype=object),
 array([0, 1, 1, 0, 1, 0, 0, 0]),
 array([1, 1]))

### Escalar datos

Esta parte se realiza para que todos los datos se puedan comparar de una manera optima entre sí, por ejemplo las columnas de valores que tienen cantidades grandes como es el sueldo, necesitan ser comparadas matematicamente con las columnas de valores pequeños como la edad. Para ello se escalan los datos, es decir, se les aplica una transformación para que todos los datos estén en la misma escala.

In [39]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler() # Create a standard scaler object
X_train = sc.fit_transform(X_train) # Fit the scaler on the training data and transform it
X_test = sc.transform(X_test) # Transform the testing data using the same scaler

pd.DataFrame(X_train)

,0,1,2,3,4
0,-0.774597,1.732051,-0.774597,1.456818,1.607827
1,1.290994,-0.577350,-0.774597,1.195570,1.280117
2,1.290994,-0.577350,-0.774597,-0.502539,-0.440360
3,-0.774597,-0.577350,1.290994,-0.009071,-0.931925
4,-0.774597,-0.577350,1.290994,-1.547529,-1.259635
5,1.290994,-0.577350,-0.774597,0.673075,0.706625
6,-0.774597,-0.577350,1.290994,-0.110667,-0.194578
7,-0.774597,1.732051,-0.774597,-1.155657,-0.768070


En este caso, no estandarizaremos nuestro vector columna $y$ que representa la variable indpendiente puesto que este algoritmo es para una categorización binaria y no es necesario. En caso de que fuera una categorización múltiple, si que sería necesario estandarizarlo.